In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import zipfile

# Откройте zip-архив
with zipfile.ZipFile('archive.zip', 'r') as zip_ref:
    # Прочитайте файлы в память
    with zip_ref.open('sample_submission.csv') as file:
        df = pd.read_csv(file)
    with zip_ref.open('train.csv') as file:
        train = pd.read_csv(file)
    with zip_ref.open('test.csv') as file:
        test = pd.read_csv(file)

### Датасет train

In [ ]:
print('Dataset size', train.shape)

num_rows, num_feature = train.shape

print('row number: ', num_rows)
print('feature number: ', num_feature)
print()
print('names of features: ', list(train))

Dataset size (38740, 3)
row number:  38740
feature number:  3

names of features:  ['oid', 'category', 'text']


In [ ]:
vc = train['category'].value_counts()
vc

autosport       3160
extreme         3110
martial_arts    3050
motosport       3030
boardgames      3020
tennis          3000
esport          2990
athletics       2970
hockey          2950
volleyball      2950
football        2860
basketball      2850
winter_sport    2800
Name: category, dtype: int64

In [ ]:
rslt_train = train.sample(frac=0.15, replace=True, random_state=1) # отбор 15% строк от исходного датасета, чтобы быстрее обучалось

In [ ]:
print(rslt_train.head(10))

             oid      category  \
33003  711989171  winter_sport   
12172  638513466  martial_arts   
5192   318047892        esport   
32511  580113489     autosport   
7813   602662466        tennis   
21440  472506655    volleyball   
32912  227965168    basketball   
20609   94534649        esport   
7751   441546566    boardgames   
31228  282153752  winter_sport   

                                                    text  
33003  Лыжная сборная перед Олимпиадой 2018 Будучи со...  
12172  Наталья появилась на прошедшем выпуске NXT и а...  
5192   Привет 33 Хотим представить вам новый киберспо...  
32511  GTWortokenoid 9 этап Валенсия. Квалификация 2....  
7813   Петра Квитова на пресс конференции в преддвери...  
21440  История которая стала хитом Ярославль Комус Фе...  
32912  Ну и главная новость Малик Монк покидает Лейке...  
20609  За шо. . . . Ивент китайского коллекционера це...  
7751   Уже совсем скоро ждем вторую комикс игру серии...  
31228  Двукратный олимпийский чемпи

In [ ]:
# востановим индексы
newdf =rslt_train.reset_index()
print('Dataset size', newdf.shape)
num_rows, num_feature =newdf.shape

Dataset size (5811, 4)


In [ ]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 60.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=9e12d9aa7951a6b2c1a545aa6c5e7f2e66e0cc4d688f01f246e90b529f78e716
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pymorphy2
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

import time
from tqdm import tqdm
import re

# Обработка текстов
morph = pymorphy2.MorphAnalyzer()

start = time.time()

y_data = []

for i in tqdm(range(num_rows)):
  s = str(newdf['text'][i])
  s1 = re.sub(r"[,.?“/!@#$1234567890#—ツ►๑۩۞۩•*”˜˜”*°°*`)(]", '', s)

  tokens = word_tokenize(s1)
  doc_stem = ''
  for j in range(0, len(tokens)):
    tokens[j] = morph.parse(tokens[j])[0].normal_form
    doc_stem = doc_stem + tokens[j] + ' '

  y_data.append(doc_stem)

  stop = time.time()

print('time of execution (sec)', stop - start)


100%|██████████| 5811/5811 [02:17<00:00, 42.13it/s]

time of execution (sec) 137.93104028701782


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
words = stopwords.words("russian")

print('Vectorization of documents')
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(ngram_range=(1,1),stop_words=(words))

data = vec.fit_transform(y_data).toarray()

num_docs, num_features = data.shape
print('documents number : ', num_docs, 'feature number: ', num_features)

Vectorization of documents
documents number :  5811 feature number:  26352


In [ ]:
import pickle
with open('vectorizer15.pk', 'wb') as fin:
    pickle.dump(vec, fin)

In [ ]:
import pickle
vectorizer = pickle.load(open("vectorizer15.pk", "rb"))
data = vectorizer.fit_transform(y_data).toarray()
num_docs, num_feature =data.shape
print('documents number : ', num_docs, 'feature number: ', num_feature)

documents number :  5811 feature number:  26352


In [ ]:
newdata = pd.DataFrame(data=data)
print(newdata.shape)
newdata = pd.concat([newdata, newdf['category']], axis=1)

In [ ]:
newdata['category'].value_counts()

autosport       485
martial_arts    470
volleyball      457
motosport       454
esport          451
athletics       451
tennis          446
extreme         441
basketball      436
boardgames      435
winter_sport    434
football        427
hockey          424
Name: category, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
Y = newdata['category'].values
X = newdata.drop(columns='category')

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print('Test collection size: ', X_test.shape)
print('Training collection size: ', X_train.shape)

Test collection size:  (1163, 26352)
Training collection size:  (4648, 26352)


1. RandomForestClassifier

In [ ]:
# создаем диапазоны чисел для параметров

# Number of trees in random forest
n_estimators = [100, 200]
# Maximum number of levels in tree
max_depth = [5, 7, 10]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

In [ ]:
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
import time
from sklearn.ensemble import RandomForestClassifier

start = time.time()
# define RF with some  parameters
myRF = RandomForestClassifier(
    n_estimators=50,
    criterion='gini',
    max_depth=5,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='log2',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    n_jobs=-1,
    random_state=0,
    verbose=0,
    warm_start=False,
    class_weight='balanced')

# define type of cross validation
mycv = ShuffleSplit(n_splits=2, test_size=0.3, random_state=0)

clf = GridSearchCV(myRF, random_grid, cv=mycv)
clf.fit(X_train, y_train)

# getting best result
print(clf.best_params_)
print(clf.best_score_)
stop = time.time()
# results is set lematized documents
print('time of execution (sec)', stop - start)

{'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
0.6351254480286739
time of execution (sec) 241.56179475784302


In [ ]:
import pickle

with open("model_up_RF15.pkl", "wb") as f:
    pickle.dump(clf, f)

In [ ]:
my_label=clf.predict(X_test)

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

print(metrics.classification_report(y_test, my_label, zero_division=0))

              precision    recall  f1-score   support

   athletics       0.73      0.73      0.73        84
   autosport       0.82      0.66      0.73        95
  basketball       0.74      0.71      0.73       103
  boardgames       0.59      0.85      0.70        88
      esport       0.58      0.34      0.43        97
     extreme       0.32      0.66      0.43        80
    football       0.72      0.55      0.63        76
      hockey       0.62      0.62      0.62        80
martial_arts       0.76      0.41      0.53        95
   motosport       0.75      0.74      0.75        93
      tennis       0.80      0.88      0.84        95
  volleyball       0.78      0.68      0.73        88
winter_sport       0.69      0.70      0.69        89

    accuracy                           0.66      1163
   macro avg       0.69      0.66      0.66      1163
weighted avg       0.69      0.66      0.66      1163



2. KNeighborsClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV

knn = KNeighborsClassifier()

start = time.time()
param_grid = [ {'n_neighbors': [1, 3, 7], 'weights': ['uniform','distance'], 'p': [1,2]} ]

mycv = ShuffleSplit(n_splits=2, test_size=0.3, random_state=0)
clf = GridSearchCV(knn, param_grid, cv=mycv , scoring='f1_macro')

clf.fit(X_train, y_train)

# getting best result
print(clf.best_params_)
print(clf.best_score_)
stop = time.time()
# results is set lematized documents
print('time of execution (sec)', stop - start)

{'n_neighbors': 7, 'p': 2, 'weights': 'distance'}
0.7387712559827849
time of execution (sec) 2087.6733961105347


In [ ]:
my_label=clf.predict(X_test)

In [ ]:
# сравним полученный результат с известными ответами
# вывод качества построенной модели
print(metrics.classification_report(y_test, my_label, zero_division=0))
# create confusion matrix
print(metrics.confusion_matrix(y_test, my_label))

              precision    recall  f1-score   support

   athletics       0.69      0.81      0.74        84
   autosport       0.89      0.76      0.82        95
  basketball       0.76      0.79      0.78       103
  boardgames       0.88      0.88      0.88        88
      esport       0.72      0.65      0.68        97
     extreme       0.58      0.56      0.57        80
    football       0.71      0.67      0.69        76
      hockey       0.75      0.75      0.75        80
martial_arts       0.79      0.64      0.71        95
   motosport       0.78      0.77      0.78        93
      tennis       0.84      0.95      0.89        95
  volleyball       0.66      0.84      0.74        88
winter_sport       0.82      0.79      0.80        89

    accuracy                           0.76      1163
   macro avg       0.76      0.76      0.76      1163
weighted avg       0.76      0.76      0.76      1163

[[68  1  1  0  4  0  1  1  2  1  2  1  2]
 [ 3 72  2  2  2  3  1  0  0  5  0  5

In [ ]:
# сохранение обученной модели в файл
import pickle

with open("my_modelKNN15.pkl", "wb") as f:
    pickle.dump(clf, f)

3. SVC

In [ ]:
from sklearn import datasets
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV

start = time.time()
# define RF with some  parameters
random_grid = [
  {'C': [0.1, 10, 100], 'kernel': ['linear']},
  {'C': [10, 100], 'kernel': ['rbf']},
 ]

mySVM = SVC(gamma='scale')
# define type of cross validation
mycv = ShuffleSplit(n_splits=2, test_size=0.3, random_state=0)

clf = GridSearchCV(mySVM, random_grid, cv=mycv)
clf.fit(X_train, y_train)

# getting best result
print(clf.best_params_)
print(clf.best_score_)
stop = time.time()
# results is set lematized documents
print('time of execution (sec)', stop - start)

{'C': 10, 'kernel': 'linear'}
0.7860215053763441
time of execution (sec) 6123.19221830368


In [ ]:
my_label=clf.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, my_label, zero_division=0))

              precision    recall  f1-score   support

   athletics       0.88      0.80      0.84        84
   autosport       0.89      0.87      0.88        95
  basketball       0.91      0.78      0.84       103
  boardgames       0.93      0.89      0.91        88
      esport       0.69      0.78      0.73        97
     extreme       0.42      0.74      0.53        80
    football       0.74      0.82      0.78        76
      hockey       0.92      0.71      0.80        80
martial_arts       0.74      0.69      0.72        95
   motosport       0.90      0.80      0.85        93
      tennis       0.99      0.92      0.95        95
  volleyball       0.87      0.78      0.83        88
winter_sport       0.84      0.82      0.83        89

    accuracy                           0.80      1163
   macro avg       0.82      0.80      0.81      1163
weighted avg       0.83      0.80      0.81      1163



In [ ]:
import pickle

with open("my_modelSVM15.pkl", "wb") as f:
    pickle.dump(clf, f)

### Датасет test

In [ ]:
print('Dataset size', test.shape)

num_rows, num_feature = test.shape

print('row number: ', num_rows)
print('feature number: ', num_feature)
print()
print('names of features: ', list(test))

Dataset size (26260, 2)
row number:  26260
feature number:  2

names of features:  ['oid', 'text']


In [ ]:
# Обработка текстов

morph = pymorphy2.MorphAnalyzer()

start = time.time()

y_data1 = []

for i in tqdm(range(num_rows)):
    s = str(test['text'][i])
    s1 = re.sub(r"[,.?“/!@#$1234567890#—ツ►๑۩۞۩•*”˜˜”*°°*`)(]", '', s)

    tokens = word_tokenize(s1)
    doc_stem=''
    for j in range(0,len(tokens)):
        tokens[j] = morph.parse(tokens[j])[0].normal_form
        doc_stem = doc_stem + tokens[j] + ' '

    y_data1.append(doc_stem)

stop = time.time()
print('time of execution (sec)', stop - start)

100%|██████████| 26260/26260 [07:17<00:00, 60.04it/s]

time of execution (sec) 437.37292075157166


In [ ]:
print('Vectorization of documents')

data1 = vectorizer.transform(y_data1).toarray()

num_docs, num_feature =data1.shape
print('documents number : ', num_docs, 'feature number: ', num_feature)

Vectorization of documents
documents number :  26260 feature number:  26352


Так как SVM дал наивысший accuracy, делаем предсказания с помощью него. Также можно отметить, что все классы по отдельности тоже предсказываются хорошо.

In [ ]:
with open("my_modelSVM15.pkl", "rb") as f:
    modelSVM = pickle.load(f)

In [ ]:
predictedSVM = modelSVM.predict(data1)

In [ ]:
test

,oid,text
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...
1,452466036,Естественное восстановление после тяжелой трен...
2,161038103,Тема нарядов продолжается Одна из британских ж...
3,663621910,Привет Избранный. Ты спрашиваешь себя ЧТО здес...
4,566255305,КОРОЛЬ ПЯТИСОТНИКОВ В ДЕЛЕ Андрей Рублев успеш...
...,...,...
26255,169728316,Выиграй коллекционный пазл по Wortokenoid of W...
26256,279369911,Волейбол от первого лица Егора Пупынина переко...
26257,600699419,Вы были когда нибудь на свидании где вам задав...
26258,560223506,ТОП 20 самых эффективных общефизических упражн...


In [ ]:
import copy
submit = copy.deepcopy(test)

submit=submit.drop(['text'], axis=1)

submit['category'] = predictedSVM

In [ ]:
submit

,oid,category
0,749208109,esport
1,452466036,martial_arts
2,161038103,extreme
3,663621910,esport
4,566255305,tennis
...,...,...
26255,169728316,boardgames
26256,279369911,volleyball
26257,600699419,boardgames
26258,560223506,martial_arts


In [ ]:
submit.to_csv('submission_file.csv', index=False)